### Install some libs

!pip install treelib

!pip3 install tree_sitter

### Import some libs

In [1]:
from treelib import Node, Tree
from tree_sitter import Language, Parser
import os
import numpy as np

### Initialize Tree-sitter go parser

In [2]:
Language.build_library(
  # Store the library in the `build` directory
  'build/my-languages.so',

  # Include one or more languages
  [
    'tree-sitter-go'
  ]
)

GO_LANGUAGE = Language('build/my-languages.so', 'go')

parser = Parser()
parser.set_language(GO_LANGUAGE)

## Step I : Find the Feature Toggles (FT)

In [24]:
keywords = ["connectThrashCheck", "foldersInProfileTab", "inviteFriends", "tabletSupport", 
            "moveOrCopy", "newTeamBuildingForChatAllowMakeTeam", "teamInvites", "teamsRedesign", 
            "webOfTrust", "whyDidYouRender", "FeatureFlags"]
# removing "admin" because it affects about 100 files 
# and it is tricky to detect if it is really the feature flag that is involved

# adding FeatureFlags because some featureflags might be accessed through the table of feature flags

### List files with keywords

#### List all .go files

In [25]:
go_folders = [x[0] for x in os.walk("./client-master/")]

go_files = []
for dir_name in go_folders:
    files = [dir_name+"/"+k for k in os.listdir(dir_name) if k[len(k)-3:] ==".go"]
    go_files.extend(files)

print("10 random .go files in this project :")
for k in range(10):
    print(go_files[np.random.randint(len(go_files))])

10 random .go files in this project :
./client-master/go/vendor/github.com/go-sql-driver/mysql/buffer.go
./client-master/go/vendor/github.com/keybase/colly/http_backend.go
./client-master/go/contacts/phonebook_test.go
./client-master/go/kbfs/dokan/winacl/security_information.go
./client-master/go/kbfs/libgit/autogit_manager_test.go
./client-master/go/vendor/golang.org/x/sys/unix/cap_freebsd.go
./client-master/go/vendor/github.com/blevesearch/bleve/index/scorch/mergeplan/sort.go
./client-master/go/vendor/github.com/dustin/go-humanize/comma.go
./client-master/go/citogo/s3.go
./client-master/go/kbfs/libkbfs/subscription_manager_test.go


In [26]:
len(go_files)

4744

#### List the interesting files, containing at least one keyword with the featureflag 

In [27]:
go_file_interests = []

for file in go_files:
    s = ""
    with open(file, "r") as f:
        s+=f.read()+"\n"
    kw_file = [k for k in keywords if k in s]
    if len(kw_file) > 0:
        print(file, "contains :")
        for kw in kw_file:
            print("->",kw)
        go_file_interests.append(file)
        print("\n")

./client-master/go/stellar/stellarsvc/service.go contains :
-> FeatureFlags


./client-master/go/keybase/main.go contains :
-> FeatureFlags


./client-master/go/engine/features_test.go contains :
-> FeatureFlags


./client-master/go/libcmdline/cmdline.go contains :
-> FeatureFlags


./client-master/go/protocol/keybase1/notify_users.go contains :
-> webOfTrust


./client-master/go/protocol/keybase1/invite_friends.go contains :
-> inviteFriends


./client-master/go/externals/services.go contains :
-> FeatureFlags


./client-master/go/teams/chain.go contains :
-> teamInvites


./client-master/go/teams/hidden/hidden.go contains :
-> FeatureFlags


./client-master/go/chat/journey_card_manager.go contains :
-> FeatureFlags


./client-master/go/libkb/chain_link_v2.go contains :
-> FeatureFlags


./client-master/go/libkb/test_common.go contains :
-> FeatureFlags


./client-master/go/libkb/features.go contains :
-> FeatureFlags


./client-master/go/libkb/globals.go contains :
-> FeatureFlags




In [28]:
len(go_file_interests)

21

## Step 3 - Search Feature Toggles dependencies in the AST

In [29]:
for gfi in go_file_interests:
    
    s = ""
    with open(gfi, "r") as f:
        s+=f.read()+"\n"

    print("File :            ", gfi, "\n")

    source = bytes(s, "utf8")
    ast = parser.parse(source)

    root_node = ast.root_node

    tree = Tree()

    root_node = ast.root_node

    type_nodes = dict()

    def get_name(node):
        return source[node.start_byte:node.end_byte].decode('utf8')

    def get_id(node):
        global type_nodes
        node_type = node.type
        if node_type not in type_nodes:
            type_nodes[node_type]=1
        else:
            type_nodes[node_type]+=1
        return node_type+str(type_nodes[node_type])

    def process(root_id, node):
        global tree
        node_id = get_id(node)
        node_name = get_name(node)
        for kw in keywords:
            if kw in node_name and node.type == 'if_statement':
                print("\n\n"+get_name(node)+"\n\n")
                for c in node.children:
                    if c.type == 'binary_expression' or c.type == 'unary_expression' or c.type == 'call_expression':
                        print("->",get_name(c))
        tree.create_node(node_name,
                         node_id,
                         parent = root_id)
        if len(node.children) != 0:
            for i in range(len(node.children)):
                process(node_id, node.children[i])

    tree.create_node("root", "root")
    for i in range(len(root_node.children)):
        process("root", root_node.children[i])
    print("\n")

    #tree.show()

File :             ./client-master/go/stellar/stellarsvc/service.go 



File :             ./client-master/go/keybase/main.go 



if !e.GetFeatureFlags().Admin(e.GetUID()) {
		// Admin only for now
		return
	}


-> !e.GetFeatureFlags().Admin(e.GetUID())


File :             ./client-master/go/engine/features_test.go 



File :             ./client-master/go/libcmdline/cmdline.go 



File :             ./client-master/go/protocol/keybase1/notify_users.go 



File :             ./client-master/go/protocol/keybase1/invite_friends.go 



File :             ./client-master/go/externals/services.go 



File :             ./client-master/go/teams/chain.go 



if prevState.IsImplicit() {
			// Check to see if the additions were previously members of the team
			checkImpteamInvites := func() error {
				addedUIDs := make(map[keybase1.UID]bool)
				for _, invites := range additions {
					for _, invite := range invites {
						cat, err := invite.Type.C()
						if err != nil {
							return err
	

In [22]:
type_nodes

{'package_clause': 1,
 'package': 1,
 'package_identifier': 71,
 '\n': 279,
 'import_declaration': 1,
 'import': 1,
 'import_spec_list': 1,
 '(': 199,
 'import_spec': 30,
 'interpreted_string_literal': 173,
 '"': 346,
 ')': 199,
 'var_declaration': 15,
 'var': 15,
 'var_spec': 15,
 'identifier': 628,
 '=': 30,
 'expression_list': 208,
 'function_declaration': 19,
 'func': 27,
 'parameter_list': 36,
 'parameter_declaration': 54,
 'type_identifier': 166,
 'pointer_type': 39,
 '*': 42,
 'qualified_type': 67,
 '.': 257,
 'block': 55,
 '{': 172,
 'short_var_declaration': 77,
 ',': 422,
 ':=': 81,
 'call_expression': 159,
 'selector_expression': 190,
 'field_identifier': 364,
 'argument_list': 159,
 'int_literal': 36,
 'return_statement': 11,
 'return': 11,
 '}': 172,
 'composite_literal': 60,
 'literal_value': 109,
 'keyed_element': 174,
 ':': 174,
 'slice_type': 22,
 '[': 48,
 ']': 48,
 'element': 93,
 'unary_expression': 18,
 '&': 14,
 'comment': 65,
 'assignment_statement': 9,
 'binary_e

In [14]:
# Pattern matching -> does not work yet

query = GO_LANGUAGE.query("""
(function_definition
  name: (identifier) @function.def)

(call
  function: (identifier) @function.call)
""")

captures = query.captures(tree.root_node)

